# Registration example using multiscale-imaging
This notebook contains an example of applying the registration with the functions transferred into the multiscale-imaging package.

In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

import multiscale.microscopy.ome as ome
import multiscale.LINK_system.coordinate as coord
import multiscale.itk.transform as tran
import multiscale.itk.itk_plotting as iplt

# Set up registration
First define the paths to each needed file.  This includes the transform parameters, the images themselves, as well as the US position list and the first tile of the microscopy image.

In [2]:
# The input files are available at https://uwmadison.box.com/s/05ou19dsi2y2wfzgocf1ab7cbbeb4fgn
fiducial_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\Fiducial')

mouse_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\For registration')

registered_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\Registered')

In [3]:
# l22_path = Path(mouse_dir, 'L22 Mouse 1596 Z16727.tif')
l22_path = Path(mouse_dir, 'L22 Mouse 210 Z16869 75dB.tif')

l22_pl_path = Path(mouse_dir, 'L22 Mouse 210 Z16869.pos')
l22_param_path = Path(mouse_dir, 'L22 Mouse 210 Z16869_Run-1_Settings.mat')
l22_transform_path = Path(fiducial_dir, 'L22 Transform.txt')

Now open the image.  If the spacing information is not properly written to either image it can still be set manually.

In [4]:
l22_spacing = [18, 18, 18]
l22_gauge_height = 16869
l22_image = coord.open_us(l22_path, l22_pl_path, l22_param_path, l22_spacing, l22_gauge_height)


Load the coordinate transform

In [5]:
microscopy_path = Path(mouse_dir, 'SHG down 8x.tif')
microscopy_tile_path = Path(mouse_dir, 'SHG settings.ome.tif')
microscopy_image = coord.open_microscopy(microscopy_path, microscopy_tile_path, downsample_factor=8)

In [6]:
with open(l22_transform_path, 'rb') as fp:
    l22_transform = pickle.load(fp)
    print(l22_transform)

[-3342.78975828  5146.56978679 24273.31049018]


Apply the registration and save the results

In [7]:
l22_to_l38_reg_path = Path(registered_dir, 'L22 to L38.tif')

In [8]:
l22_to_shg_reg = tran.apply_transform_params(microscopy_image, l22_image, 
                                     l22_transform,
                                     sitk.TranslationTransform(3, [0, 0, 0]))

# QUS parameter registration

In [27]:
analysis_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\L22 Analysis 0-25 latwindow')
l22_ibc_path = Path(analysis_dir, 'L22 Analysis 0-25 latwindow_IBC.tif')
l22_ibc_spacing = [50, 100, 18]
dynamic_range = 60
l22_ibc = coord.open_us(l22_ibc_path, l22_pl_path, l22_param_path, l22_ibc_spacing, l22_gauge_height, dynamic_range)

In [28]:
l22_ibc_reg = tran.apply_transform_params(microscopy_image, l22_ibc, 
                                     l22_transform,
                                     sitk.TranslationTransform(3, [0, 0, 0]))

In [29]:
l22_ibc_overlay = Path(registered_dir, 'L22 IBC and SHG on SHG coords 0-25 latwindow.tif')
ome.save_ijstyle_overlay([l22_to_shg_reg[:, :, 4:20], microscopy_image[:, :, 4:20], l22_ibc_reg[:, :, 4:20]], l22_ibc_overlay)